# Imports

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
import pickle
import numpy as np
import pandas as pd
import os

# Data

In [59]:
data = [["layer1", "layer2", "layer3", "target"]]

In [60]:
def getDividedArray(array):
  '''
  Breaks longer arrays into chunks of length four
  :param array: array of layer sequence
  :return array containing chunks of length four
  :rtype: array
  '''
  temp = []
  if len(array) > 4:
    for i in range(len(array)-3):
      temp.append(array[i:i+4])
  return temp


def unpickleData(filename):
  '''
  Get data from pickled files
  :param filename: pickle file with model layer sequence
  :param data: the array containing the model array
  '''
  fileObj = open(filename,'rb')
  array = pickle.load(fileObj)
  if len(array) > 4:
    array = getDividedArray(array)
    for i in array:
      data.append(i)
  else:
    data.append(array)
  fileObj.close()
  return data

In [61]:
for file in os.listdir('../data/models'):
    filename=os.path.join('../data/models',file)
    unpickleData(filename)

In [62]:
# Column Names
# data = [["layer1", "layer2", "layer3", "target"]]

# get all data in data array
# connect to ec2 and call unpickleData


# convert it to csv
np.savetxt("../data/csv/data.csv", data, delimiter=",", fmt='%s')

In [63]:
df=pd.read_csv("../data/csv/data.csv")
df.head()

,layer1,layer2,layer3,target
0,Conv2D,Conv2D,Conv2D,MaxPooling2D
1,Conv2D,Conv2D,MaxPooling2D,Flatten
2,Conv2D,MaxPooling2D,Flatten,Dense
3,MaxPooling2D,Flatten,Dense,Dense
4,Flatten,Dense,Dense,Dense


In [64]:
layer_map={
    'Conv2D':0,
    'Dense':1,
    'LSTM':2,
    'SimpleRNN':3,
    'Dropout':4,
    'Flatten':5,
    "ZeroPadding2D":6,
    "AveragePooling2D":7,
    "MaxPooling2D":8   
}

In [9]:
def one_hot_encode(data,label_map):
    '''
    One hot encode the data according to the label map
    :param data: the data to be encoded
    :param label_map: the label map which is to be used to encode the data
    :return: the encoded data
    '''
    one_hot_encoded=list()
    for i in range(len(data)):
        temp=[0]*len(label_map)
        label=df.iloc[i].target
        temp[label_map[label]]=1
        one_hot_encoded.append(temp)
    return pd.DataFrame(one_hot_encoded,columns=label_map.keys())

In [17]:

def encode_data(data,labeL_map):
    '''
    Encode the data according to the provided label map
    :param data: the data to be encoded
    :param label_map: the label map which is to be used to encode the data
    :return: the encoded data
    '''
    data=data.copy()
    encoded_cols=['layer1','layer2','layer3']
    for col in encoded_cols:
        data[col]=data[col].apply(lambda x:labeL_map[x])
    return data

In [13]:
def split_data(data,train_percentage):
    '''
    Split the data into training and testing data
    :param data: the data to be split
    :param train_percentage: the percentage of the data to be used for training
    :return: the training and testing data
    '''
    train_data=data.sample(frac=train_percentage,random_state=1)
    test_data=data.drop(train_data.index)
    return train_data,test_data

In [28]:
def get_data(data,label_map):
    '''
    Get the data from the dataframe
    :param data: the data to be used
    :param label_map: the label map which is to be used to encode the data
    :return: the training and testing data
    '''
    df1=encode_data(data,label_map)
    df2=one_hot_encode(data,label_map)
    df1.drop(['target'],axis=1,inplace=True)
    df3=pd.concat([df1,df2],axis=1)
    train_data,test_data=split_data(df3,0.8)
    return train_data,test_data

In [65]:
train_data,test_data=get_data(df,layer_map)
X_train,y_train=train_data[['layer1','layer2','layer3']],train_data.drop(['layer1','layer2','layer3'],axis=1)
X_test,y_test=test_data[['layer1','layer2','layer3']],test_data.drop(['layer1','layer2','layer3'],axis=1)

In [36]:
X_train,y_train=np.array(X_train),np.array(y_train)
X_test,y_test=np.array(X_test),np.array(y_test)

In [66]:

# print(X_train[0],y_train[0])
print(len(X_train))

1900


# Model

Refrenece for the model
[link](https://stackoverflow.com/questions/64209325/if-i-want-to-predict-the-next-element-in-a-sequence-of-numbers-what-do-i-need-t)

For survey:
- [Making Predictions with Sequences](https://machinelearningmastery.com/sequence-prediction/)
- 

In [67]:
model = Sequential([
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(layer_map), activation='softmax'),
])

In [68]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train,batch_size=1,epochs=10)

Epoch 1/10
1900/1900 [==============================] - 1s 573us/step - loss: 1.1357 - accuracy: 0.6889
Epoch 2/10
1900/1900 [==============================] - 1s 588us/step - loss: 0.8084 - accuracy: 0.7542
Epoch 3/10
1900/1900 [==============================] - 1s 699us/step - loss: 0.6946 - accuracy: 0.7821
Epoch 4/10
1900/1900 [==============================] - 1s 583us/step - loss: 0.6680 - accuracy: 0.7889
Epoch 5/10
1900/1900 [==============================] - 1s 586us/step - loss: 0.6566 - accuracy: 0.7837
Epoch 6/10
1900/1900 [==============================] - 1s 579us/step - loss: 0.6425 - accuracy: 0.7800
Epoch 7/10
1900/1900 [==============================] - 1s 606us/step - loss: 0.6325 - accuracy: 0.7853
Epoch 8/10
1900/1900 [==============================] - 1s 725us/step - loss: 0.6192 - accuracy: 0.7947
Epoch 9/10
1900/1900 [==============================] - 1s 616us/step - loss: 0.6225 - accuracy: 0.7884
Epoch 10/10
1900/1900 [==============================] - 1s 586u

In [ ]:
model.save('../models/linear_classification_model.h5')

# Prediction and analysis

In [4]:
model=tf.keras.models.load_model('../models/linear_classification_model.h5')

In [9]:
pred=model.predict([[1,1,1]])
print(pred)

[[3.5328607e-03 9.3743759e-01 3.9375559e-03 6.4367690e-04 5.1750425e-02
  2.6787026e-03 2.2631619e-09 4.8344870e-09 1.9206931e-05]]


In [70]:
train_data.head()

,layer1,layer2,layer3,Conv2D,Dense,LSTM,SimpleRNN,Dropout,Flatten,ZeroPadding2D,AveragePooling2D,MaxPooling2D
1701,1,1,4,0,1,0,0,0,0,0,0,0
285,5,1,1,0,1,0,0,0,0,0,0,0
1179,1,0,0,0,0,0,0,0,0,0,0,1
1592,1,1,1,0,1,0,0,0,0,0,0,0
306,0,0,0,1,0,0,0,0,0,0,0,0
